In [1]:
import os
from pathlib import Path
# Change cwd to the project root (parent of 'notebooks/')
os.chdir(Path.cwd().parent)
Path.cwd()

PosixPath('/Users/jbrandt/code/birddog')

In [2]:
from birddog.core import Archive, form_element_text
from birddog.utility import fetch_url, form_text_item, convert_utc_time, get_text
from birddog.wiki import (
    history_url, 
    get_page_history, 
    get_page_history_from_cutoff, 
    HistoryLRU, ARCHIVES, 
    ARCHIVE_LIST, 
    batch_fetch_document_links)
from urllib.parse import quote
#import mwclient
from bs4 import BeautifulSoup

2025-04-18 11:59:26,640 [INFO] Using local folder /Users/jbrandt/code/birddog/.cache for storage.


In [3]:
archive = Archive('DAZHO')
archive.default_url

2025-04-18 11:59:26,647 [INFO] HistoryLRU.lookup(ДАЖО/Д): cache miss
2025-04-18 11:59:26,788 [INFO] fetch_url: 1 requests in last 60s → 0.02 req/s
2025-04-18 11:59:26,792 [INFO] Retrieved from cache: DAZHO-D[2025,01,19,12:47]: page_cache/DAZHO-D/2025,01,19,12:47.json


'https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94'

In [4]:
archive.url

'https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94'

In [5]:
archive.unquoted_url

'https://uk.wikisource.org/wiki/Архів:ДАЖО/Д'

In [6]:
get_page_history(archive.title, limit=5)

[{'revid': 871572,
  'modified': '2025,01,19,12:47',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94&oldid=871572'},
 {'revid': 819407,
  'modified': '2024,11,20,15:03',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94&oldid=819407'},
 {'revid': 634322,
  'modified': '2023,09,11,21:31',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94&oldid=634322'},
 {'revid': 632048,
  'modified': '2023,09,02,21:34',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94&oldid=632048'},
 {'revid': 632022,
  'modified': '2023,09,02,21:09',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/%D0%94&oldid=632022'}]

In [7]:
fond = archive.lookup('1')
fond.title

2025-04-18 11:59:26,970 [INFO] HistoryLRU.lookup(ДАЖО/1): cache miss
2025-04-18 11:59:27,125 [INFO] Loading page: DAZHO-D/1 from https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%96%D0%9E/1


'ДАЖО/1'

In [8]:
get_page_history(fond.title, limit=5)

[{'revid': 771001,
  'modified': '2024,09,12,08:41',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/1&oldid=771001'},
 {'revid': 769635,
  'modified': '2024,09,09,14:03',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/1&oldid=769635'},
 {'revid': 767861,
  'modified': '2024,09,06,09:10',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/1&oldid=767861'},
 {'revid': 767857,
  'modified': '2024,09,06,09:04',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/1&oldid=767857'},
 {'revid': 691693,
  'modified': '2024,03,17,21:33',
  'link': 'https://uk.wikisource.org/w/index.php?title=%D0%90%D1%80%D1%85%D1%96%D0%B2%3A%D0%94%D0%90%D0%96%D0%9E/1&oldid=691693'}]

In [9]:
from urllib.parse import unquote

In [10]:
get_page_history('xyz')

2025-04-18 11:59:27,665 [ERROR] get_page_history(xyz, limit=10): unrecognized page name


[]

In [11]:
opus = archive.lookup('1').lookup('1')

2025-04-18 11:59:27,678 [INFO] HistoryLRU.lookup(ДАЖО/1): cache hit
2025-04-18 11:59:27,683 [INFO] Retrieved from cache: DAZHO-D/1[2024,09,12,08:41]: page_cache/DAZHO-D/1/2024,09,12,08:41.json
2025-04-18 11:59:27,683 [INFO] HistoryLRU.lookup(ДАЖО/1/1): cache miss
2025-04-18 11:59:27,835 [INFO] Loading page: DAZHO-D/1/1 from https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%96%D0%9E/1/1


In [12]:
kids = [f"{opus.title}/{get_text(child[0]['text'])}" for child in opus.children if 'redlink' not in child[0]['link']]

In [13]:
batch_fetch_document_links(kids[:1])

{'ДАЖО/1/1/43': ['/wiki/File:ДАЖО_1-1-43._1796._По_донесению_духовных_правлений_о_умертвии_священено-_и_церковнослужителей.pdf']}

In [14]:
batch_fetch_document_links(kids * 10)

{'ДАЖО/1/1/111': ['/wiki/File:ДАЖО_1-1-111._1795_По_репорту_Благочиннаго_Протоиерея_Иоанна_Бочковскаго_о_худыхь_поступкахь_Староконстантиновскаго_уезда_села_Багл.pdf'],
 'ДАЖО/1/1/119': ['/wiki/File:ДАЖО_1-1-119._1795_По_доношению_Протоиерея_Андрея_Колтоновскаго_о_бытии_ему_на_Воздвиженскомь_города_Полона.pdf'],
 'ДАЖО/1/1/198': ['/wiki/File:ДАЖО_1-1-198._1796._О_дозволении_священническому_сыну_Иоакиму_Блонскому_приискать_праздное_священническое_место.pdf'],
 'ДАЖО/1/1/376': ['/wiki/File:ДАЖО_1-1-376._1796._О_присоединении_к_православию_церквей_и_приходских_людей_в_Владимирском_уезде.pdf'],
 'ДАЖО/1/1/43': ['/wiki/File:ДАЖО_1-1-43._1796._По_донесению_духовных_правлений_о_умертвии_священено-_и_церковнослужителей.pdf']}

In [15]:
for child in opus.children:
    if child[1].get('link'):
        print(child[0]['text']['uk'], child[1])

In [16]:
opus.load_child_document_links()

2025-04-18 11:59:28,835 [INFO] load_child_document_links(DAZHO-D/1/1) updating cache


In [17]:
for child in opus.children:
    if child[1].get('link'):
        print(child[0]['text']['uk'], child[1])

43 {'text': {'uk': 'По донесению духовных правлений о умертвии священено- и церковнослужителей'}, 'link': '/wiki/File:ДАЖО_1-1-43._1796._По_донесению_духовных_правлений_о_умертвии_священено-_и_церковнослужителей.pdf'}
111 {'text': {'uk': 'По репорту Благочиннаго Протоиерея Иоанна Бочковскаго о худыхь поступкахь Староконстантиновскаго уезда села Баглаевь священника Григория Прокоповича вь блудодеянии сь служанкою его Феклою Ковалевою и о лишении его Прокоповича священства'}, 'link': '/wiki/File:ДАЖО_1-1-111._1795_По_репорту_Благочиннаго_Протоиерея_Иоанна_Бочковскаго_о_худыхь_поступкахь_Староконстантиновскаго_уезда_села_Багл.pdf'}
119 {'text': {'uk': 'По доношению Протоиерея Андрея Колтоновскаго о бытии ему на Воздвиженскомь города Полона приходе и о набдении на оной открытымь указом и о проч.'}, 'link': '/wiki/File:ДАЖО_1-1-119._1795_По_доношению_Протоиерея_Андрея_Колтоновскаго_о_бытии_ему_на_Воздвиженскомь_города_Полона.pdf'}
198 {'text': {'uk': 'О дозволении священническому сыну Иоаки

In [18]:
opus = archive.lookup('393').lookup('1')

2025-04-18 11:59:28,853 [INFO] HistoryLRU.lookup(ДАЖО/393): cache miss
2025-04-18 11:59:28,992 [INFO] Loading page: DAZHO-D/393 from https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%96%D0%9E/393
2025-04-18 11:59:29,129 [INFO] HistoryLRU.lookup(ДАЖО/393/1): cache miss
2025-04-18 11:59:29,274 [INFO] Loading page: DAZHO-D/393/1 from https://uk.wikisource.org/wiki/%D0%90%D1%80%D1%85%D1%96%D0%B2:%D0%94%D0%90%D0%96%D0%9E/393/1


In [19]:
opus.load_child_document_links()

2025-04-18 11:59:30,021 [INFO] load_child_document_links(DAZHO-D/393/1) updating cache


In [20]:
for child in opus.children:
    if child[1].get('link'):
        print(child[0]['text']['uk'], child[1])

1 {'text': {'uk': 'Аржанцев Тимофей'}, 'link': '/wiki/File:ДАЖО_393-1-1_Аржанцев_Тимофей_(1851).pdf'}
2 {'text': {'uk': 'Каливарский Янкель'}, 'link': '/wiki/File:ДАЖО_393-1-2_Каливарский_Янкель_(1852).pdf'}
3 {'text': {'uk': 'Малятер Зельман'}, 'link': '/wiki/File:ДАЖО_393-1-3_Малятер_Зельман_(1850).pdf'}
4 {'text': {'uk': 'Спектор Хаим'}, 'link': '/wiki/File:ДАЖО_393-1-4_Спектор_Хаим_(1851).pdf'}
5 {'text': {'uk': 'Годовой отчет Житомирского казенного училища 1-го разряда училища'}, 'link': '/wiki/File:ДАЖО_393-1-5_Годовой_отчет_Житомирского_казенного_училища_1-го_разряда_училища_(1850).pdf'}
6 {'text': {'uk': 'Программы и штаты в училищах 1-го разряда'}, 'link': '/wiki/File:ДАЖО_393-1-6_Программы_и_штаты_в_училищах_1-го_разряда_(1850).pdf'}
7 {'text': {'uk': 'О перемене директоров в училищах Волынской губ.'}, 'link': '/wiki/File:ДАЖО_393-1-7_О_перемене_директоров_в_училищах_Волынской_губ._(1850).pdf'}
8 {'text': {'uk': 'Ведомости о поведении и успеваемости учеников 1-го класса'}, 'l

In [21]:
opus.load_child_document_links()